In [1]:
import re
import os
import json
import glob
from collections import Counter

from tqdm import tqdm
import pandas as pd
import Levenshtein as lev
import numpy as np

import rispy
from pybtex.database import parse_string
from pylatexenc.latex2text import LatexNodes2Text
conv = LatexNodes2Text()

In [2]:
def extract_isbn(input_string):
    """
    Extract an ISBN from an unstructured text string.
    """
    # Regular expression to match "ISBN:" followed by any space-free combination of digits and hyphens.
    pattern = r'ISBN:?\s*([\d-]+)'
    
    match = re.search(pattern, input_string)
    if match:
        return match.group(1)  # Return the matched part (ISBN number)
    else:
        return None  # No ISBN found following the "ISBN:" prefix

In [3]:
def map_book(ris, bibt):
    # extract ISBN for abstract field, if available:
    if 'abstract' in ris:
        abstract = ris['abstract']
        isbn = extract_isbn(abstract.strip())
        if isbn:
            ris['issn'] = isbn

    # make editors authors, if applicable:
    if 'editor' in bibt.persons:
        if 'author' in bibt.persons:
            ris['first_authors'] = [conv.latex_to_text(str(editor)) for editor in bibt.persons['author']]
        else:
            if 'author' in ris:
                del ris['author']
            if 'authors' in ris:
                del ris['authors']
        ris['secondary_authors'] = [conv.latex_to_text(str(editor)) for editor in bibt.persons['editor']]

    if 'first_authors' in ris and 'secondary_authors' in ris:
        if 'author' in ris:
            del ris['author']
        if 'authors' in ris:
            del ris['authors']

    # add translators
    if 'translator' in bibt.fields:
        ris['tertiary_authors'] = conv.latex_to_text(bibt.fields['translator']).split(' and ')
        if 'author' in ris:
            del ris['author']
        if 'first_authors' in ris:
            del ris['first_authors']

    for f in bibt.fields:
        bibt.fields[f] = conv.latex_to_text(bibt.fields[f])
    
    # replace non-distinct title with parsed journal title:
    if 'title' in bibt.fields and bibt.fields['title'].split():
        ris['notes_abstract'] = ris['title']
        ris['title'] = bibt.fields['title']
    elif 'booktitle' in bibt.fields and bibt.fields['booktitle'].split():
        ris['notes_abstract'] = ris['title']
        ris['title'] = bibt.fields['booktitle']
    
    if 'pagetotal' in bibt.fields:
        ris['end_page'] = bibt.fields['pagetotal']
    
    if 'pages' in bibt.fields:
        ris['end_page'] = bibt.fields['pages']
    
    if 'publisher' in bibt.fields:
        ris['publisher'] = bibt.fields['publisher']

    if 'place' in bibt.fields:
        ris['place_published'] = bibt.fields['place']

    if 'place' in bibt.fields:
        ris['place_published'] = bibt.fields['place']

    if 'address' in bibt.fields:
        ris['place_published'] = bibt.fields['address']
    
    if 'location' in bibt.fields:
        ris['place_published'] = bibt.fields['location']

    if 'series' in bibt.fields:
        if ';' in bibt.fields['series']:
            series, vol = [e.strip() for e in bibt.fields['series'].split(';', maxsplit=1)]
            ris['secondary_title'] = series
            ris['volume'] = vol
        else:
            ris['secondary_title'] = bibt.fields['series']

    if 'volume' in bibt.fields:
        ris['volume'] = bibt.fields['volume']

    if 'number' in bibt.fields:
        ris['number'] = bibt.fields['number']

    if 'edition' in bibt.fields:
        ris['edition'] = bibt.fields['edition']
    
    return ris

In [4]:
def map_chapter(ris, bibt):
    # extract ISBN for abstract field, if available:
    if 'abstract' in ris:
        abstract = ris['abstract']
        isbn = extract_isbn(abstract.strip())
        if isbn:
            ris['isbn'] = isbn

    # make editors authors, if applicable:
    if 'editor' in bibt.persons:
        if 'author' in bibt.persons:
            ris['first_authors'] = [conv.latex_to_text(str(editor)) for editor in bibt.persons['author']]
        else:
            if 'authors' in ris:
                del ris['authors']
        ris['secondary_authors'] = [conv.latex_to_text(str(editor)) for editor in bibt.persons['editor']]

    if 'first_authors' in ris and 'secondary_authors' in ris:
        if 'authors' in ris:
            del ris['authors']

    # add translators
    if 'translator' in bibt.fields:
        ris['tertiary_authors'] = conv.latex_to_text(bibt.fields['translator']).split(' and ')

    for f in bibt.fields:
        bibt.fields[f] = conv.latex_to_text(bibt.fields[f])
    
    # replace non-distinct title with parsed journal title:
    if 'title' in bibt.fields:
        ris['notes_abstract'] = ris['title']
        ris['title'] = bibt.fields['title']

    if 'booktitle' in bibt.fields:
        ris['secondary_title'] = bibt.fields['booktitle']
    
    if 'pagetotal' in bibt.fields:
        ris['end_page'] = bibt.fields['pagetotal']
    
    if 'pages' in bibt.fields:
        pages = bibt.fields['pages'].split('-')
        if len(pages) == 2:
            ris['start_page'] = pages[0]
            ris['end_page'] = pages[1]
        else:
            ris['end_page'] = bibt.fields['pages']
    
    if 'publisher' in bibt.fields:
        ris['publisher'] = bibt.fields['publisher']

    if 'place' in bibt.fields:
        ris['place_published'] = bibt.fields['place']

    if 'address' in bibt.fields:
        ris['place_published'] = bibt.fields['address']
    
    if 'location' in bibt.fields:
        ris['place_published'] = bibt.fields['location']

    if 'series' in bibt.fields:
        ris['tertiary_title'] = bibt.fields['series']

    if 'volume' in bibt.fields:
        ris['volume'] = bibt.fields['volume']

    if 'number' in bibt.fields:
        ris['number'] = bibt.fields['number']

    if 'edition' in bibt.fields:
        ris['edition'] = bibt.fields['edition']
    
    return ris

Collect already available normalizations for journal titles (so that we can find out below which one we miss):

In [5]:
jtitle = pd.read_excel('../data/journal_titles_master.xlsx')
existing_jtitles = set(jtitle['normalized'])
lower2jtitles = dict(zip(jtitle['normalized'].str.lower(), jtitle['normalized']))
jtitle.head(30)

,secondary_title,normalized,count,issn
0,Ons erfdeel: kultureel tijdschrift voor Zuidvl...,Ons erfdeel,2660.0,0030-2651
1,Dietsche warande en Belfort: tijdschrift voor ...,Dietsche warande en Belfort,2461.0,0012-2645
2,De nieuwe taalgids: tweemaandelijks tijdschrif...,De nieuwe taalgids,2359.0,0028-9922
3,Bzzlletin; Stichting BZZTôH Teater. Voorburg: ...,Bzzlletin,1638.0,0165-0858
4,Poëziekrant: tweemaandelijks tijdschrift. Gent...,Poëziekrant,1573.0,2030-0638
5,Onze taal: maandblad van het Genootschap Onze ...,Onze taal,1322.0,0165-7828
6,Vlaanderen: tweemaandelijks tijdschrift voor k...,Vlaanderen,1312.0,0042-7683
7,De gids: nieuwe vaderlandsche letteroefeningen...,De gids,1249.0,0016-9730
8,Levende talen: berichten en mededelingen van d...,Levende talen,1239.0,0024-1539
9,Tijdschrift voor Nederlandse taal- en letterku...,Tijdschrift voor Nederlandse taal- en letterkunde,962.0,0040-7550


In [6]:
def map_journal(ris, bibt):
    """
    Merges the newly structured information in the bibtex returned
    by the LLM into the already available RIS entry from the dump.
    Reliably structured information (e.g. authors, year, keywords, ...)
    from the RIS entries is maximally retained.
    """
    #print(ris)
    #print(bibt)
    #print('===============================================')

    for f in bibt.fields:
        bibt.fields[f] = conv.latex_to_text(bibt.fields[f])
    
    # replace unstrucuted title with parsed journal title (if available):
    if 'title' in bibt.fields:
        # keep track of original title description:
        ris['notes_abstract'] = ris['title']
        ris['title'] = bibt.fields['title']
        if ris['title'].strip().lower() in ('in', 'untitled', 'title of the article', 'title of the article (if provided)'):
            ris['title'] = ''
    
    # parse pagination information:
    if 'pagetotal' in bibt.fields:
        ris['end_page'] = bibt.fields['pagetotal']
    if 'pages' in bibt.fields:
        pages = bibt.fields['pages'].split('-')
        if len(pages) == 2:
            ris['start_page'] = pages[0]
            ris['end_page'] = pages[1]
        else:
            ris['end_page'] = bibt.fields['pages']

    # collect parsed journal title (unless we had that information already, which will be more reliable)
    if 'secondary_title' not in ris and 'journal' in bibt.fields:
        journal = bibt.fields['journal']
        # sometimes place of publication of the journal is added: we remove that
        journal = journal.split('(')[0].strip()
        journal = journal.split('[')[0].strip()
        ris['journal_name'] = journal
    elif 'secondary_title' in ris and 'journal' in bibt.fields:
        ris['journal_name'] = ris['secondary_title']
        del ris['secondary_title']

    if 'journal_name' in ris:
        jn = ris['journal_name']
        if jn.startswith('"') and jn.endswith('",'):
            jn = jn[1:-2]
        if jn.count('"') == 1:
            jn = jn.replace('"', '')
        ris['journal_name'] = jn
    
    # collect information on volume and issue
    if 'volume' in bibt.fields:
        ris['volume'] = bibt.fields['volume']
    if 'number' in bibt.fields:
        ris['number'] = bibt.fields['number']
    if 'number' not in bibt.fields and 'issue' in bibt.fields:
        ris['number'] = bibt.fields['issue']
    
    return ris

In [7]:
map_entry = {
             'JOUR': map_journal,
             'CHAP': map_chapter,
             'BOOK': map_book,
            }

In [8]:
def deduplicate_bibtex(bibt):
    """
    Deduplicate repeated fields in the bibtex returned by the LLM.
    We only keep the first appearance of a given field.
    """
    lines, fields = [], set()
    for line in bibt.strip().split('\n'):
        if line.startswith('@') or line == '}':
            lines.append(line)
        else:
            field = line.split('=')[0].strip()
            if field not in fields:
                lines.append(line)
                fields.add(field)

    clean = '\n'.join([l for l in lines if l])
    if not clean.strip().endswith('}'):
        clean += '\n}\n'
    
    return clean


def clean_bibtex(bibt):
    """
    Attempts to correct some common syntactic errors in the bibtex
    returned by the LLM (which cause the pybtex parser to fail).
    """
    if not bibt:
        return ''
    
    # remove erroneous markdown syntax:
    bibt = bibt.replace('```bibtex', '').replace('```', '').replace("```tex", '')
    
    # sometimes mutliple bibtexs are created: we only keep the first one
    bibt = [b for b in bibt.split('@') if b.strip()]
    bibt = '@' + bibt[0]

    lines = []
    for line in bibt.strip().split('\n'):
        l = line.strip()

        # take care of spaces in the bibtex key:
        if l.startswith('@') and ' ' in l:
            line = ''.join(line.split())
        
        # fix missing entry keys:
        if l in ('@article{,', '@article{'):
            lines.append('@article{xxx,')
            continue

        # common errors:
        if l.endswith(']'):
            line += '},'
            lines.append(line)
            continue
        line = line.replace('{ )', '{}')
        if l.endswith("',"):
            line = line[:-2] + '},'
        
        # ensure that end-of-line syntax is respected:
        if l != '}':
            if not l.endswith('},'):
                if l.endswith('}'):
                    line += ','
                elif not l.endswith('}') and not l.endswith(','):
                    line += '},'
            if l.endswith('),'):
                line = line.replace('),', ')},')
        
        # add missing curly brackets:
        if '=' in l and (not '{' in l or not '}' in l):
            k, v = [e.strip() for e in l.split('=')][:2]
            v.replace(',', '')
            line = '  ' + k + '=' + '{' + v + '},'
        
        # correct curly bracket syntax in title field:
        if l.startswith('title') and l.count('}') > 1:
            k, v = [e.strip() for e in l.split('=')][:2]
            v = v.replace('{', '').replace('}', '')
            line = '  ' + k + '=' + '{' + v + '},'

        lines.append(line)

    # recompose the lines of the bibtex entry:
    clean = '\n'.join([l for l in lines if l])
    if not clean.strip().endswith('}'):
        clean += '\n}\n'
    
    # return the deduplicated version of the bibtex entry:
    return deduplicate_bibtex(clean)

In [9]:
d = """@article{smit1951,
  author = {Van Duinkerken, Anton},
  title = {Over: Ternauwernood},
  journal = {Critisch Bulletin},
  volume = {18},
  number = {10},
  month = {October},
  year = {1951},
  pages = {462-467},
  publisher = {Utrecht [etc.]
}"""
print(clean_bibtex(d))
assert parse_string(clean_bibtex(d), 'bibtex')

@article{smit1951,
  author = {Van Duinkerken, Anton},
  title = {Over: Ternauwernood},
  journal = {Critisch Bulletin},
  volume = {18},
  number = {10},
  month = {October},
  year = {1951},
  pages = {462-467},
  publisher = {Utrecht [etc.]},
}


In [10]:
s = """@article{,
  title={Uitreiking Taaluniepenning 1991},
  author={unknown},
  journal={Publikatieblad; Nederlandse Taalunie},
  year={1992},
  volume={23},
  number={jan},
  pages={1-9}
}"""

print(clean_bibtex(s))
assert parse_string(clean_bibtex(s), 'bibtex')

@article{xxx,
  title={Uitreiking Taaluniepenning 1991},
  author={unknown},
  journal={Publikatieblad; Nederlandse Taalunie},
  year={1992},
  volume={23},
  number={jan},
  pages={1-9},
}


In [11]:
llm_path = '../data/llm-dump'

new_jtitles = Counter()

for decade_folder in sorted(glob.glob(f'{llm_path}/*')):
    #if '1990s' not in decade_folder:
    #        continue
    print(':::', decade_folder, ':::')

    for spreadsheet_path in sorted(glob.glob(f'{decade_folder}/*.xlsx')):
        df = pd.read_excel(spreadsheet_path, header=0, engine='openpyxl')
        #n = 1000
        #if len(df) > n:
        #    df = df.sample(n)

        if 'bibtex' not in df.columns:
            continue
    
        ptype = os.path.basename(spreadsheet_path).replace('.xlsx', '')
        print('     - ', spreadsheet_path, f'({ptype})')

        if ptype != 'JOUR':
            continue
        
        # parse the RIS (stored as JSON strings in the spreadsheet)
        df['RIS'] = df['RIS'].apply(json.loads)

        # clean (and deduplicate the bibtex returned by the LLM)
        cleaned = []
        for bt in df['bibtex']:
            if isinstance(bt, str):
                cleaned.append(clean_bibtex(bt))
            else:
                cleaned.append('')
        df['bibtex-clean'] = cleaned

        # Update the available RIS entries with newly structure info,
        # returned by the LLM (and keep tracked of whether or not that is successful):
        updated_ris, status = [], []
        for ris, bibtex_str in tqdm(list(zip(df['RIS'], df['bibtex-clean']))):
            if isinstance(bibtex_str, str):
                try:
                    #print(bibtex_parse)
                    bibtex_parse = parse_string(bibtex_str, 'bibtex')
                    single_key = list(bibtex_parse.entries.keys())[0]
                    updated = map_entry[ptype](ris.copy(), bibtex_parse.entries[single_key])

                    # keep track of new journal titles which lack a normalized variant,
                    # (unless the difference is only in capitalization):
                    if ptype == 'JOUR' and 'journal_name' in updated and updated['journal_name'] not in existing_jtitles:
                        try:
                            updated['journal_name'] = lower2jtitles[updated['journal_name'].lower()]
                        except KeyError:
                            new_jtitles[updated['journal_name']] += 1
                    
                    updated['label'] = 'success'
                    updated_ris.append(updated)
                    status.append('success')
                except Exception as e:
                    print(e)
                    ris['label'] = f'failure ({str(e)})'
                    updated_ris.append(ris)
                    status.append('failure')
            else:
                ris['label'] = 'failure'
                updated_ris.append(ris)
                status.append('failure')

        # store the newly merged information as a JSON string that holds a RIS entry:
        df['consolidated'] = [json.dumps(r, indent=2, ensure_ascii=False) for r in updated_ris]
        df['status'] = status

        # re-encode the original RIS entry as a JSON string in the original column:
        df['RIS'] = [json.dumps(d, indent=2, ensure_ascii=False) for d in df['RIS']]

        # remove the cleaned bibtex string:
        del df['bibtex-clean']

        # output new spreadsheet:
        df.to_excel(spreadsheet_path, index=False, header=True)

        # output updated RIS file:
        with open(f'{decade_folder}/{ptype}_consolidated.ris', 'w') as bibliography_file:
            rispy.dump(updated_ris, bibliography_file)

        # show the failure statistics:
        print(df['status'].value_counts())

::: ../data/llm-dump/1940s :::
     -  ../data/llm-dump/1940s/BOOK.xlsx (BOOK)
     -  ../data/llm-dump/1940s/CHAP.xlsx (CHAP)
     -  ../data/llm-dump/1940s/JFULL.xlsx (JFULL)
     -  ../data/llm-dump/1940s/JOUR.xlsx (JOUR)


  0%|          | 0/9897 [00:00<?, ?it/s]

list index out of range
list index out of range


  4%|▍         | 374/9897 [00:00<00:05, 1874.55it/s]

list index out of range
list index out of range


...

status
success    5236
failure       2
Name: count, dtype: int64
     -  ../data/llm-dump/2020s/WEB.xlsx (WEB)
::: ../data/llm-dump/2030s :::
     -  ../data/llm-dump/2030s/BOOK.xlsx (BOOK)
::: ../data/llm-dump/misc :::
     -  ../data/llm-dump/misc/ADVS.xlsx (ADVS)
     -  ../data/llm-dump/misc/BOOK.xlsx (BOOK)
     -  ../data/llm-dump/misc/CHAP.xlsx (CHAP)
     -  ../data/llm-dump/misc/EJOUR.xlsx (EJOUR)
     -  ../data/llm-dump/misc/JFULL.xlsx (JFULL)
     -  ../data/llm-dump/misc/JOUR.xlsx (JOUR)


 98%|█████████▊| 3419/3483 [00:02<00:00, 1659.90it/s]

syntax error in line 8: premature end of file


100%|██████████| 3483/3483 [00:02<00:00, 1673.14it/s]


status
success    3482
failure       1
Name: count, dtype: int64
     -  ../data/llm-dump/misc/WEB.xlsx (WEB)


Extract journal titles for which we don't have a normalization yet and map them provionally to the closest available normalized title (using the Levenshtein distance):

In [12]:
mappings = []
for nj, cnt in new_jtitles.items():
    distances = np.array([lev.distance(nj, oj) for oj in jtitle['normalized']])
    mappings.append([nj, cnt] + list(jtitle.iloc[np.argmin(distances)][['normalized', 'issn']]))

mappings = pd.DataFrame(mappings, columns=['raw title', 'count', 'normalized', 'issn'])
mappings = mappings.sort_values('count', ascending=False)
mappings.sample(30)

,raw title,count,normalized,issn
3031,Tydskrif vir Nederlands & Afrikaans,20,Tydskrif vir wetenskap en kuns,0372-3526
3154,Tijdschrift voor Nederlandse taal- letterk...,1,Tijdschrift voor Nederlandse taal- en letterkunde,0040-7550
289,Land van mijn hart,1,Rond Janus en Bet,NaN
2188,Verrigtinge van die vyfde driejaarlikse neerla...,2,Vereniging van Vlaamse toneelauteurs,NaN
737,Rivista di etnografia,1,Lust en gratie,0168-8413
1125,Jaarboek van de Heemkundevereniging Roerstreek,1,Blad van de Heemkundevereniging Den Dungen,NaN
3634,Nederkandsch museum,1,Nederlandse post,NaN
1879,Juffr. Ida,5,Juffrouw Ida,0927-4847
448,Leidraad voor de wereld van reklame en marketing,1,Werkblad voor Nederlandse didactiek,NaN
361,vol. 1,1,Volk,NaN


We save this spreadsheet for manual correction:

In [13]:
mappings.to_excel('../data/journal_titles_2ndBatch.xlsx', header=True, index=False)